In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# cd /content/drive/MyDrive/OML---mini-project/src

In [ ]:
# !pip install -r requirements.txt

In [ ]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import torch.optim.lr_scheduler as schedulers
from utils import retrieve_setup, retrieve_training_params, training_loop

In [5]:
model_name = "MobileNetV3Small" 
dataset_name = "CIFAR10"
lr_max = 3e-4 # achived in step 2150
lr_min = 5e-5 # achived in step 2150
n_epochs = 8

In [7]:
# retrieves
model, dataset = retrieve_setup(model_name, dataset_name)
optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "parameters.yml")

# change lr from default to lr_max
for g in optimizer.param_groups:
    g["lr"] = lr_max

# implement linear increasing scheduler
total_iters = int(len(dataset["train"])/batch_size*n_epochs)
start_factor = lr_min/lr_max

scheduler_name, scheduler = "rangeLR_pretrained", schedulers.LinearLR(
    optimizer, 
    start_factor=start_factor,
    end_factor=1,
    total_iters=total_iters
)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# run training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_, metrics = training_loop(
    model=model,
    dataset=dataset,
    scheduler=scheduler,
    optimizer=optimizer,
    loss_fn=loss_fn,
    n_epochs=n_epochs,
    batch_size=batch_size,
    train_strategy=("iter", 25),
    test_strategy=("iter", 25),
    scheduler_strategy="iter",
    file_name=f"/{dataset_name}_{scheduler_name}",
    device=device,
)